In [ ]:
import pandas as pd
import numpy as np
from random import randint


def generate_location_data(patient_count: int, location_count: int) -> pd.DataFrame:
    row_count = patient_count * 10
    patient_ids = np.random.randint(1, patient_count + 1, size=row_count)
    locations = np.array([str(chr(randint(65, 65 + patient_count - 1))) for i in range(row_count)])
    df = pd.DataFrame()
    df['ptid'] = pd.Series(patient_ids)
    df['loc'] = pd.Series(locations)
    return df.sort_values(by='ptid').reset_index(drop=True)

def get_transfers(data: pd.DataFrame) -> pd.DataFrame:
    df = data.copy()
    df['from_loc'] = df['loc']
    df['to_ptid'] = pd.Series(np.roll(df['ptid'].values, -1))
    df['to_loc'] = pd.Series(np.roll(df['loc'].values, -1))
    df = df[df['ptid'] == df['to_ptid']]
    return df.drop(columns=['loc', 'to_ptid'])

In [ ]:
locations = generate_location_data(20, 5)
transfers = get_transfers(locations)

In [ ]:
tx = transfers.copy()
tx['transfer'] = tx['from_loc'] + ' -> ' + tx['to_loc']
tx = tx.drop(columns=['from_loc', 'to_loc'])
tx_sorted = tx.sort_values(by=['ptid', 'transfer']).reset_index(drop=True)
tx_sorted['count'] = pd.Series(np.ones(shape=len(tx_sorted)))
tx_grouped = tx_sorted.groupby(by=['ptid', 'transfer']).count()

In [ ]:
tx_matrix = tx_grouped.unstack(level=1, fill_value=0)
tx_matrix.columns = tx_matrix.columns.droplevel(0)
tx_matrix